In [ ]:
rm(list = ls())
setwd("/home/creambbq/facu/Datos de panel/TP4")
library("haven"); library("plm"); library("dplyr"); library("pglm");
data <- read_dta('keane.dta')

#Solo hombres negros y creo la variable employ rezagada un periodo 
data <- data %>% 
  filter(black == 1) %>% 
  group_by(id) %>% 
  mutate(employ_lag = dplyr::lag(employ))

### Ejercicio 2.A

In [4]:
pdata <- pdata.frame(data, index = c("id", "year")) #Equiv. a declarar el panel en Stata
probit <- pglm(employ ~ employ_lag, 
               family = binomial("probit"), 
               model = "pooling", 
               method = "bfgs",
               data = pdata)
summary(probit)

--------------------------------------------
Maximum Likelihood estimation
BFGS maximization, 21 iterations
Return code 0: successful convergence 
Log-Likelihood: -2282.448 
2  free parameters
Estimates:
            Estimate Std. error t value Pr(> t)    
(Intercept) -0.54008    0.02795  -19.32  <2e-16 ***
employ_lag   1.38717    0.04340   31.96  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
--------------------------------------------

### Ejercicio 2.B 

In [5]:
#P[employ_it=1|employ_it-1=1] = pnorm(delta + rho*1)
pnorm(probit$estimate[1]+probit$estimate[2])



(Intercept) 
  0.8015263

In [6]:
#P[employ_it=1|employ_it-1=0] = pnorm(delta)
pnorm(probit$estimate[1])

(Intercept) 
  0.2945704

### Ejercicio 2.C

In [7]:
probit2 <- pglm(employ ~ employ_lag +  y83 + y84 + y85 + y86 + y87, 
                family = binomial("probit"), 
                model = "pooling", 
                method = "bfgs",
                data = pdata)
summary(probit2)

--------------------------------------------
Maximum Likelihood estimation
BFGS maximization, 48 iterations
Return code 0: successful convergence 
Log-Likelihood: -2248.844 
7  free parameters
Estimates:
            Estimate Std. error t value  Pr(> t)    
(Intercept) -0.88271    0.05479 -16.109  < 2e-16 ***
employ_lag   1.31813    0.04503  29.272  < 2e-16 ***
y83          0.34210    0.07403   4.621 3.81e-06 ***
y84          0.45179    0.07473   6.046 1.49e-09 ***
y85          0.52448    0.07589   6.911 4.82e-12 ***
y86          0.39079    0.07677   5.090 3.57e-07 ***
y87          0.52861    0.07673   6.889 5.61e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
--------------------------------------------

In [8]:
#P[employ_i87=1|employ_it-1=1] = pnorm(delta + rho*1 + d87)
pnorm(probit$estimate[1]+probit$estimate[2]+probit2$estimate[7])

(Intercept) 
  0.9155418

In [9]:
#P[employ_it=1|employ_it-1=0] = pnorm(delta + rho*0 + d87)
pnorm(probit$estimate[1]+probit2$estimate[7])

(Intercept) 
   0.495423

### Ejercicio 2.D


In [10]:
data <- data %>% group_by(id) %>% mutate(employ_81 = sum(employ*y81))
pdata <- pdata.frame(data, index = c("id", "year")) # Defino el panel de nuevo
probit3 <- pglm(employ ~ employ_lag + employ_81 + y83 + y84 + y85 + y86 + y87, 
                family = binomial("probit"), 
                model = "random", 
                method = "bfgs",
                data = pdata)
summary(probit3)

--------------------------------------------
Maximum Likelihood estimation
BFGS maximization, 63 iterations
Return code 0: successful convergence 
Log-Likelihood: -2208.745 
9  free parameters
Estimates:
            Estimate Std. error t value  Pr(> t)    
(Intercept) -1.00344    0.06526 -15.376  < 2e-16 ***
employ_lag   0.89149    0.06722  13.262  < 2e-16 ***
employ_81    0.57189    0.08812   6.490 8.60e-11 ***
y83          0.43415    0.07946   5.464 4.67e-08 ***
y84          0.65075    0.08327   7.815 5.49e-15 ***
y85          0.79825    0.08784   9.088  < 2e-16 ***
y86          0.68967    0.08947   7.708 1.27e-14 ***
y87          0.84014    0.09046   9.287  < 2e-16 ***
sigma        0.55854    0.05491  10.173  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
--------------------------------------------

### Ejercicio 2.F

In [11]:
delta <- probit3$estimate[1] 
rho <- probit3$estimate[2]
eps_81 <- probit3$estimate[3]
gamma_87 <- probit3$estimate[8] #Coeficiente de dummy de tiempo 
sigma_de <- (summary(probit3)$estimate[9,1]) #idéntico al sigma_u que reporta STATA en Probit RE

In [12]:
data %>% ungroup() %>%
  filter(year == 87) %>% 
  mutate(step1 = delta + 
         rho + 
         eps_81*employ_81+
         gamma_87, 
         step2 = step1/sqrt(1+sigma_de^2), 
         pm = pnorm(step2)) %>% 
  summarise(p = mean(pm))

p
<dbl>
0.7676615


In [13]:
data %>% ungroup() %>%
  filter(year == 87) %>% 
  mutate(step1 = delta + 
           eps_81*employ_81+
           gamma_87, 
         step2 = step1/sqrt(1+sigma_de^2), 
         pm = pnorm(step2)) %>% 
  summarise(p = mean(pm))

p
<dbl>
0.4872999
